### DNN + BatchNormalization

In [7]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)


2.5.0
sys.version_info(major=3, minor=9, micro=1, releaselevel='final', serial=0)
matplotlib 3.4.2
numpy 1.19.5
pandas 1.3.1
sklearn 0.24.2
tensorflow 2.5.0
tensorflow.keras 2.5.0


In [8]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [9]:
# x = (x - u) / std

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# x_train: [None, 28, 28] -> [None, 784]
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)


In [10]:
# tf.keras.models.Sequential()

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
for _ in range(20):
    model.add(keras.layers.Dense(100, activation="relu"))
    # 可用selu激活函数替代
    model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(10, activation="softmax"))

model.compile(loss="sparse_categorical_crossentropy",
              optimizer = "adam",
              metrics = ["accuracy"])

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_21 (Dense)             (None, 100)               78500     
_________________________________________________________________
batch_normalization_20 (Batc (None, 100)               400       
_________________________________________________________________
dense_22 (Dense)             (None, 100)               10100     
_________________________________________________________________
batch_normalization_21 (Batc (None, 100)               400       
_________________________________________________________________
dense_23 (Dense)             (None, 100)               10100     
_________________________________________________________________
batch_normalization_22 (Batc (None, 100)              

In [12]:
# Tensorboard, earlystopping, ModelCheckpoint
logdir = './dnn-callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,
                                 "fashion_mnist_model.h5")

callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file,
                                    save_best_only = True),
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3),
]
history = model.fit(x_train_scaled, y_train, epochs=10,
                    validation_data=(x_valid_scaled, y_valid),
                    callbacks = callbacks)

2021-08-06 03:33:48.955929: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-06 03:33:48.955951: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.
2021-08-06 03:33:48.956855: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.


Epoch 1/10


2021-08-06 03:33:50.179866: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


   2/1719 [..............................] - ETA: 4:02 - loss: 3.6424 - accuracy: 0.0938 

2021-08-06 03:33:50.960840: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session initializing.
2021-08-06 03:33:50.960853: I tensorflow/core/profiler/lib/profiler_session.cc:141] Profiler session started.


...

2021-08-06 03:33:51.215001: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2021-08-06 03:33:51.221792: I tensorflow/core/profiler/lib/profiler_session.cc:159] Profiler session tear down.
2021-08-06 03:33:51.228575: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./dnn-callbacks/train/plugins/profile/2021_08_06_03_33_51
2021-08-06 03:33:51.231035: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to ./dnn-callbacks/train/plugins/profile/2021_08_06_03_33_51/jackydemacbook.trace.json.gz
2021-08-06 03:33:51.241722: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: ./dnn-callbacks/train/plugins/profile/2021_08_06_03_33_51
2021-08-06 03:33:51.242123: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for memory_profile.json.gz to ./dnn-callbacks/train/plugins/profile/2021_08_06_03_33_51/jackydemacbook.memory

1719/1719 [==============================] - ETA: 0s - loss: 1.3697 - accuracy: 0.5366  

2021-08-06 03:36:01.357940: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1719/1719 [==============================] - 135s 78ms/step - loss: 1.3697 - accuracy: 0.5366 - val_loss: 0.6534 - val_accuracy: 0.7566
Epoch 2/10
1719/1719 [==============================] - 136s 79ms/step - loss: 0.8037 - accuracy: 0.7220 - val_loss: 0.5627 - val_accuracy: 0.8044

KeyboardInterrupt: 

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 3)
    plt.show()

plot_learning_curves(history)

# 1. 参数众多，训练不充分
# 2. 梯度消失 -> 链式法则 -> 复合函数f(g(x))

In [ ]:
model.evaluate(x_test_scaled, y_test)